<a href="https://colab.research.google.com/github/willey95/EtCetra/blob/master/DART_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np 
import openpyxl as xl
pd.set_option('display.max_rows', 20)

####  File Upload

In [2]:
# from google.colab import files
# uploaded = files.upload()

# for fn in uploaded.keys():
#     print("Filename : {fn}")

#### Google Drive 연결하여 회사코드 추출 (CORPCODE.xml)
* 회사코드는 다운로드 받을 때 zip 형태로 되어 있음
* 압축 해제 시 xml로 되어 있어 parsing 해서 사용 

In [3]:
tree = et.parse('drive/My Drive/CORPCODE.xml')
root = tree.getroot()

#### 회사명 추출

In [4]:
rows = []
rowDic = {}

for i in root:
    corp_code = i.find("corp_code").text
    corp_name = i.find("corp_name").text
    stock_code = i.find('stock_code').text
    modify_date = i.find('modify_date').text

    rows.append ([corp_code, corp_name, stock_code, modify_date])
    rowDic[corp_code] = [corp_name, stock_code, modify_date]

df = pd.DataFrame(rows, columns=['corp_code', 'corp_name', 'stock_code', 'modify_data'])

#### 함수작성

In [5]:
def find_corp_code(co_name):
    find_df = df[df['corp_name'].str.contains(co_name, na=False)]
    return find_df

In [6]:
def load_data(request, corp_code, **kwargs):

    '''
    request : 'company 회사개요, biz_report 정기보고' \n
    corp_code : '회사명 추출, find_corp_code'로 알아냄 \n

    **kwargs \n
    
    '''

    api_key = '58e750d240b32c943d6bdcf8d58f746880655281'
    base_url = 'https://opendart.fss.or.kr/api/'
    
    result = []
    for co_item in corp_code['corp_code']:
        try : 
            if request == 'company':
                action_url = 'company.json?crtfc_key={}&corp_code={}'.format(api_key, co_item)
            
            elif request == 'biz_rpt_sum':
                bsns_year = kwargs['year'] ### 해당연도
                reprt_code = kwargs['rpt_code'] ### 보고서 코드
                
                action_url = 'fnlttSinglAcnt.json?crtfc_key={}&corp_code={}&bsns_year={}&reprt_code={}' \
                            .format(api_key,co_item,bsns_year,reprt_code)
                print(action_url)

            url = base_url+action_url
            rq = requests.get(url)
            temp = rq.json() 
            result.append (temp)
            
        except:
            print('error')
            
    return result

In [7]:
### 분기별, 반기별, 연도별 사업보고서가 달라 종류에 따라 칼럼명을 교체

def ext_Col (df) :
    rpt_code = df.loc[0, 'reprt_code']
    if rpt_code == '11011' :
        column_ext = ['account_nm','thstrm_amount','frmtrm_amount','bfefrmtrm_amount','ord']
        column_nm  = {'account_nm':'계정명',
                      'thstrm_amount' : df.loc[0, 'thstrm_dt'], 
                      'frmtrm_amount' : df.loc[0, 'frmtrm_dt'],
                      'bfefrmtrm_amount': df.loc[0, 'bfefrmtrm_dt']
                     }
    elif rpt_code == '11012' :
        column_ext = ['account_nm','thstrm_amount','frmtrm_amount','ord']
        column_nm  = {'account_nm':'계정명',
                      'thstrm_amount' : df.loc[0, 'thstrm_dt'], 
                      'frmtrm_amount' : df.loc[0, 'frmtrm_dt']  
                     }
    else :
        column_ext = ['account_nm','thstrm_amount','ord']
        column_nm  = {'account_nm':'계정명',
                      'thstrm_amount' : df.loc[0, 'thstrm_dt']  
                     }

    return column_ext, column_nm


In [8]:
def ext_FS (data, consol, kindof):

    '''
    data : JSON Data
    consol=True : 연결제무제표,  False : 별도 제무제표  

    '''
    srcWord = ''
    lstDf = []

    #재무제표의 종류
    if kindof == 'BS' :
        srcWord = '재무상태표'
    elif kindof == 'PL' :
        srcWord = '손익계산서'

    for dtTemp in data :
        if dtTemp['status'] == '000' :
            tmp_df = pd.DataFrame(dtTemp['list'])
            column_ext, column_nm = ext_Col(tmp_df)  #추출할 칼럼과 이름 바꿀 칼럼

            if consol == True :
                df = tmp_df.loc[(tmp_df['fs_nm']== '연결재무제표')&(tmp_df['sj_nm']==srcWord), column_ext]
            elif consol == False :
                df = tmp_df.loc[(tmp_df['fs_nm']== '재무제표')&(tmp_df['sj_nm']==srcWord), column_ext]

            df['ord'] = df['ord'].astype('int')
            df = df.sort_values(by='ord', ascending=True).reset_index()

            df.rename(columns=column_nm, inplace=True)

        else :
            df = pd.DataFrame()     #빈 데이터 프레임을 생성하여 회사개수와 일치시킴
        
        lstDf.append (df)

    return lstDf

In [9]:
def ch_rpt_code (str) :
    if str == '1Q' :
        str_r = '11013'

    elif str == '2Q' :
        str_r = '11012'

    elif str == '3Q' :
        str_r = '11014'

    elif str == '4Q' :
        str_r = '11011'
    
    return str_r

In [10]:
corp_code = find_corp_code("삼성전자")

# 보고서 종류 기준연도 등 입력 
input_data = {'year' : '2019',              #문자형으로 입력
              'rpt_code' : '4Q',            #'1Q, 2Q, 3Q, 4Q'
              'kind_of_report' : 'BS'       #'BS' or 'PL'
             }
corp_code

,corp_code,corp_name,stock_code,modify_data
45194,00126380,삼성전자,005930,20191209
53846,00252074,삼성전자판매,,20200217
56775,00258999,삼성전자서비스,,20200225
58695,01345812,삼성전자서비스씨에스,,20200219
70495,00366997,삼성전자로지텍,,20200324


#### 회사 Profile

In [11]:
co_profile = load_data(request='company', corp_code=corp_code)

#### 재무정보 추출

In [12]:
input_data['rpt_code'] = ch_rpt_code(input_data['rpt_code'])

temp = load_data(request='biz_rpt_sum', corp_code=corp_code, year=input_data['year'], rpt_code = input_data['rpt_code'] )   #JSON 형태 반환
lstDf = ext_FS(temp, consol=True, kindof=input_data['kind_of_report'])

for i, df in enumerate(lstDf):
    print("co_code : {}, co_name : {}".format(corp_code.iat[i,0], corp_code.iat[i,1]))
    file_name = str(corp_code.iat[i,1]) + str(input_data['kind_of_report']) + str(input_data['rpt_code']) + '.xlsx'
    
    df.to_excel(file_name)

fnlttSinglAcnt.json?crtfc_key=58e750d240b32c943d6bdcf8d58f746880655281&corp_code=00126380&bsns_year=2019&reprt_code=11011
fnlttSinglAcnt.json?crtfc_key=58e750d240b32c943d6bdcf8d58f746880655281&corp_code=00252074&bsns_year=2019&reprt_code=11011
fnlttSinglAcnt.json?crtfc_key=58e750d240b32c943d6bdcf8d58f746880655281&corp_code=00258999&bsns_year=2019&reprt_code=11011
fnlttSinglAcnt.json?crtfc_key=58e750d240b32c943d6bdcf8d58f746880655281&corp_code=01345812&bsns_year=2019&reprt_code=11011
fnlttSinglAcnt.json?crtfc_key=58e750d240b32c943d6bdcf8d58f746880655281&corp_code=00366997&bsns_year=2019&reprt_code=11011
co_code : 00126380, co_name : 삼성전자
co_code : 00252074, co_name : 삼성전자판매
co_code : 00258999, co_name : 삼성전자서비스
co_code : 01345812, co_name : 삼성전자서비스씨에스
co_code : 00366997, co_name : 삼성전자로지텍
